Extension

In [ ]:
pip install ortools

# Dataset
## Fichier d'instance de problème

Le fichier d'instance est le fichier se trouvant dans Dataset data.json

## Code de génération aléatoire

In [ ]:
import random
from datetime import datetime, time, timedelta
import math
import sys
import json

# Creation de la matrice d edistance
def compute_euclidean_distance_matrix(locations):
    """Creates callback to return distance between points."""
    distances = [[0 for x in range(len(locations))]
                 for y in range(len(locations))]
    for from_counter, from_node in enumerate(locations):
        for to_counter, to_node in enumerate(locations):
            if from_counter == to_counter:
                distances[from_counter][to_counter] = 0
            else:
                # Euclidean distance
                distances[from_counter][to_counter] = (int(
                    math.hypot((from_node[0] - to_node[0]),
                               (from_node[1] - to_node[1]))))
    return distances

# creation des coordonée de localisation
def createCoordinates(tailleListe) :
    

    dataset = {}
    dataset["location"] = list()

    increment = 0

    while increment < tailleListe : 
        coordinates = (
            random.randint(10,10000), random.randint(10,10000))
      
        dataset["location"].append(coordinates)
        increment = increment + 1

   
    return(dataset)

#  Creation de la taille des objet recuperer
def createObject(dataset, tailleListe) :
    dataset["timeWindow"] = list()

    fermeture = 0
    ouverture = 1

    increment = 0

    while increment < tailleListe : 
        ouverture = random.randint(0,20)

        dataset["timeWindow"].append(ouverture)

        fermeture = 0
        ouverture = 1
        increment = increment + 1

    return(dataset)

# Creation de la dimmensions des camion
def creationDim(dataset, listSize) :
    dataset["vehicle_capacities"] = list()
    increment = 0

    while increment < listSize : 
        dimmension = random.randint(10,100)
        dataset["vehicle_capacities"].append(dimmension)
        increment = increment + 1

    return(dataset)

# Creation des variable a utiliser pour le DataSet
def creationFonc(listSize, trucks) :

    data = createCoordinates(listSize)
    data = createObject(data, listSize)
    data = creationDim(data, trucks)
    data['location'] = compute_euclidean_distance_matrix(
        data['location'])

    return data


def main(argv):
    filename = argv[0]
    listSize = int(input("Entrez le nombre de villes : "))
    trucks= int(float(25/100) * listSize)
    print (trucks)
    data1=creationFonc(listSize, trucks)
    data = {}
    data['distance_matrix'] = data1['location']
    data['demands'] = data1['timeWindow']
    data['vehicle_capacities'] = data1['vehicle_capacities']
    data['num_vehicles'] = trucks
    data['depot'] = 0
  
    #   Insertion des donnée dans un fichier JSON
    with open("Dataset/Data.json".format(filename), 'w') as json_file:
        json.dump(data, json_file, indent=2)
  
    # On demare la fonction 
if __name__ == '__main__':
    main(sys.argv[1:])

# Code Python de l'algorithme


In [ ]:
"""Capacited Vehicles Routing Problem (CVRP)."""

from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import math
import sys
import json

# On affiche les resultat de la fonctions
def print_solution(data, manager, routing, assignment):
    """Prints assignment on console."""
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(
                node_index, route_load)
            previous_index = index
            index = assignment.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(
            manager.IndexToNode(index), route_load)
        plan_output += 'Distance of the route: {}m\n'.format(
            route_distance)
        plan_output += 'Load of the route: {}\n'.format(
            route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(
        total_distance))
    print('Total load of all routes: {}'.format(
        total_load))


def main(argv):
    """Solve the CVRP problem."""
    # Get the Dataset
    with open("Dataset/Data.json".format(argv[0]), 'r') as json_file:
        data = json.load(json_file)
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(
            data['distance_matrix']), 
            data['num_vehicles'], 
            data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(
        distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(
        transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.seconds = 30
    search_parameters.log_search = True

    # Solve the problem.
    assignment = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if assignment:
        print('Nbr de vehicules {}'.format(data['num_vehicles']))
        print('Capacité des vehicules {}'.format(
            data['vehicle_capacities']))
        print_solution(data, manager, routing, assignment)
        


if __name__ == '__main__':
    main(sys.argv[1:])

# Codes pour générer l'étude expérimentale

## Code de calcul des statistiques descriptives, voire prédictives

In [45]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats as st
import matplotlib.pyplot as plt
import math
import sys
import json

def main(argv):
    print('La taille d\'un objet se situe entre 0 et 20')
    print('La contenance d\'un vehicule se situe entre 10 et 100\n')
    # On recupere le Dataset
    with open("Dataset/Data.json".format(argv[0]), 'r') as json_file:
        data = json.load(json_file)
    datademand = data['demands']
    datavehicule = data['vehicle_capacities']
    
    moyenne(datademand, datavehicule)
    mediane(datademand, datavehicule)
    quantile(datademand, datavehicule)
    variance(datademand, datavehicule)
    ecart_type(datademand, datavehicule)
    
    
    
if __name__ == '__main__':
    main(sys.argv[1:])


La taille d'un objet se situe entre 0 et 20
La contenance d'un vehicule se situe entre 10 et 100

La moyenne de la taille des objet : 9.92
La moyenne de la contenance des vehicule : 56.0 

La médianne de la taille des objet : 10.0
La médianne de la contenace des vehicule : 55.5 

Quartile Q1 de la taille des objet : 5.0
Quartile Q3 de la taille des objet : 14.0 

Quartile Q1 de la contenace des vehicule : 5.0
Quartile Q3 de la contenace des vehicule : 14.0 

L'écart interquartile de la taille des objet : 9.0
L'écart interquartile de la contenace des vehicule : 51.0 

La variance de la taille des objet : 34.67
La variance de la contenace des vehicule : 857.0 

L'écart type de la taille des objet : 5.89
L'écart type de la contenace des vehicule : 29.27 



### Moyenne
<img src = "Image/moyenne.jpg" title = "moyenne" alt = "moyenne">


In [46]:
def moyenne(datademand, datavehicule):
    datademand = np.mean(datademand)
    datavehicule = np.mean(datavehicule)
    print('La moyenne de la taille des objet :' ,round(datademand, 2))
    print('La moyenne de la contenance des vehicule :' ,round(datavehicule, 2), '\n')

### Médiane
<img src = "Image/mediane.jpg" title = "mediane" alt = "mediane">

In [47]:
def mediane(datademand, datavehicule):
    datademand = np.median(datademand)
    datavehicule = np.median(datavehicule)
    print('La médianne de la taille des objet :' ,round(datademand, 2))
    print('La médianne de la contenace des vehicule :' ,round(datavehicule, 2) , '\n')

### Variance
<img src = "Image/variance.jpg" title = "variance" alt = "variance">

In [48]:
def variance(datademand, datavehicule):
    datademand = np.var(datademand)
    datavehicule = np.var(datavehicule)
    print('La variance de la taille des objet :' ,round(datademand, 2))
    print('La variance de la contenace des vehicule :' ,round(datavehicule, 2) , '\n') 

### L'écart-type
<img src = "Image/ecart-type.jpg" title = "ecart-type" alt = "ecart-type">

In [49]:
def ecart_type(datademand, datavehicule):
    datademand = np.std(datademand)
    datavehicule = np.std(datavehicule)
    print('L\'écart type de la taille des objet :' ,round(datademand, 2))
    print('L\'écart type de la contenace des vehicule :' ,round(datavehicule, 2), '\n') 

### Quantiles


In [50]:
def quantile(datademand, datavehicule):
    Q1d = round(np.percentile(datademand, 25))
    Q3d = round(np.percentile(datademand, 75))
    Q1v = round(np.percentile(datavehicule, 25))
    Q3v = round(np.percentile(datavehicule, 75))
    datademand = np.median(datademand)
    datavehicule = np.median(datavehicule)
    print('Quartile Q1 de la taille des objet :' ,Q1d)
    print('Quartile Q3 de la taille des objet :' ,Q3d, '\n')
    print('Quartile Q1 de la contenace des vehicule :' ,Q1v)
    print('Quartile Q3 de la contenace des vehicule :' ,Q3v, '\n')
    ecart_interquartile(Q1d,Q3d, Q1v,Q3v)



### L'écart interquartile
<img src = "Image/ecart-intercartile.jpg" title = "ecart-interquartile" alt = "ecart-interquartile">

In [51]:
def ecart_interquartile(q1demand, q3demands, q1vehicule, q3vehicule):
    print('L\'écart interquartile de la taille des objet :' ,q3demands-q1demand)
    print('L\'écart interquartile de la contenace des vehicule :' ,q3vehicule-q1vehicule , '\n') 

## Script de tests de montée en charge

Ce tests est present dans le notebook de presentation